Data science Project 1: Profitable App Profiles for the App Store and Google Play Markets

By Junchen Zhao

1. This project is a portfolio type project: We want to help our developers understand what type of apps are likely to attract more users on Google play and App store through the result from this project.

2. Our goal is to successfully build this project up.

3. We have 2 data sources:
   1. googleplaystore.csv: A data set containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018.
   2. AppleStore.csv: A data set containg containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. 

First, we start by opening and exploring these 2 source datasets by creating a function named explore_data() that we can repeatedly use to print rows in a readable way.

In [39]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') #print an empty line after each row
    
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns', len(dataset[0]))

After defining our function explore_data, then, we open the 2 datasets and use the explore_data() function to show the content of these 2 datasets.

In [40]:
from csv import reader

unreaded_googleplay_data = open('googleplaystore.csv',encoding = 'utf8')
unreaded_applestore_data = open('AppleStore.csv', encoding = 'utf8')

gg = list(reader(unreaded_googleplay_data))
readed_googleplay_data = gg[1:]
googleplay_header = gg[0]

apple = list(reader(unreaded_applestore_data))
readed_applestore_data = apple[1:]
applestore_header = apple[0]

#Explore google play store data
print('Exploring the google play store data:\n')
print(googleplay_header)
explore_data(readed_googleplay_data, 0, 5)

#Explore apple store data
print('Exploring the apple store data:\n')
print(applestore_header)
explore_data(readed_applestore_data, 0, 5)


Exploring the google play store data:

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', 

In the previous step, we opened the two data sets and performed a brief exploration of the data. Before beginning our analysis, we need to make sure the data we analyze is accurate, otherwise the results of our analysis will be wrong. This means that we need to:

    1. Detect inaccurate data, and correct or remove it.
    2. Detect duplicate data, and remove the duplicates.
    
These two steps are called data cleaning.

In [41]:
#First, the error entry row is at 10472. Then, we visualize it.
print('Error Entry row:\n')
explore_data(readed_googleplay_data, 10472, 10473)

#We find, although, the entry is at 10472, the actual position is at 10473.
#Because we have the first row for the header row.
#Then we delete the row at 10473.
del readed_googleplay_data[10472]
print('Updated Entry row:\n')
explore_data(readed_googleplay_data, 10472, 10473)

Error Entry row:

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Updated Entry row:

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']




Now, we want to find the unique and duplicated apps in the app store.

In [42]:
def find_dup_unique(dataset):
    duplicate_apps = []
    unique_apps = []

    for app in dataset:
        app_name = app[0]
        if app_name in unique_apps:
            duplicate_apps.append(app_name)
        else:
            unique_apps.append(app_name)
    print('Number of duplicate apps:', len(duplicate_apps),'\n')
    print('Examples of duplicate apps:', duplicate_apps[0:15])
    
    return [duplicate_apps, unique_apps]

dup_google_app, unique_google_app = find_dup_unique(readed_googleplay_data)

Number of duplicate apps: 1181 

Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


As we can see above, based on our result, the number of duplicated apps in the google play store is 1180. Normally, we would remove the duplicate randomly. However, now we will only keep the duplicate app based the highest number of reviewers in the google app store. So, let's remove the useless duplicated apps from the dataset now.

In [43]:
print('Expected Length After remove the duplicated apps', len(readed_googleplay_data) -1181)

reviews_max = {} #app name: number of review
for app in readed_googleplay_data:
    app_name = app[0]
    n_reviews = float(app[3])
    if app_name in reviews_max and reviews_max[app_name]<n_reviews:
        reviews_max[app_name] = n_reviews
    if app_name not in reviews_max:
        reviews_max[app_name] = n_reviews

android_clean = [] # store our new cleaned data set
already_added = [] # store app names

for app in readed_googleplay_data:
    app_name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[app_name] and app_name not in already_added:
        android_clean.append(app)
        already_added.append(app_name)
print('Actual length after cleaning the dataset',len(android_clean))
        
            

Expected Length After remove the duplicated apps 9659
Actual length after cleaning the dataset 9659


As we can see above, we successfully cleaned our dataset and removed the duplicated apps.

Now, after we removed the duplicated apps from the googleplay store data set, because we want to use English for the apps we develop at the target company, we would like to only consider the apps that are directed toward an English-speaking audience. So, our current goal is to remove the apps in the dataset which are in the non-English language.

In [44]:
print('Non-English Apps example in apple store:\n')
print(readed_applestore_data[813][1])
print(readed_applestore_data[6731][1])
print('\n')
print('Non-English Apps example in google play store:\n')
print(android_clean[4412][0])
print(android_clean[7940][0])

Non-English Apps example in apple store:

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


Non-English Apps example in google play store:

中国語 AQリスニング
لعبة تقدر تربح DZ


As we can see above from the examples, we want to remove the apps in the datasets with the names such as showing above.

Normally, when we are having the English names with the specific symbols, we are talking about letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

However, the languages other than English, inside the programming idea, are the special symbols. Behind the scenes, each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character 'a' is 97, character 'A' is 65, and character '爱' is 29,233.

We can get the corresponding number of each character using the ord() built-in function.

In [45]:
#Example of getting the corresponding number which associate the symbols
print(ord('a'))
print(ord('A'))
print(ord('爱'))
print(ord('5'))
print(ord('+'))

97
65
29233
53
43


So, based on the return from the ord function, we can see the numbers corresponding to the characters we commonly use in English text are all in the range 0 to 127 according to ASCII. 

Then, we can based on the number and the range of the English alphebat according to the ASCII to determine the whether the apps name is English or not so that we can remove it from the dataset.

In [46]:
def english_or_not(name):
    num_list = []
    for char in name:
        if ord(char) not in range(0,128):
            num_list.append(ord(char))
    if len(num_list) >3:
        return False
    else:
        return True
        

print(english_or_not('Instagram'))
print(english_or_not('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_or_not('Docs To Go™ Free Office Suite'))
print(english_or_not('Instachat 😜'))

#data exploration for identifying non-English apps in each dataset
android_english =[]
apple_english = []

for app in android_clean:
    name = app[0]
    if english_or_not(name):
        android_english.append(app)
        
for app in readed_applestore_data:
    name = app[0]
    if english_or_not(name):
        apple_english.append(app)

print('Number of rows in android English is:', len(android_english))
print('Number of rows in apple English is:', len(apple_english))
        

True
False
True
True
Number of rows in android English is: 9614
Number of rows in apple English is: 7197


So far in the data cleaning process, we have done:

1. Removed inaccurate data
2. Removed duplicate app entries
3. Removed non-English apps

Now, we want to isolate the free apps in the next step of data cleaning.

In [47]:
free_google_app = []
free_apple_app = []

for app in android_english:
    if app[6] == 'Free':
        free_google_app.append(app)

for app in apple_english:
    if float(app[4]) == 0.0:
        free_apple_app.append(app)

print('The number of non-free English apps in the google play store:', len(android_english)-len(free_google_app))
print('The number of non-free English apps in the apple store:', len(apple_english)-len(free_apple_app))

The number of non-free English apps in the google play store: 751
The number of non-free English apps in the apple store: 3141


So far, we spent a good amount of time on cleaning data, and:

1. Removed inaccurate data
2. Removed duplicate app entries
3. Removed non-English apps
4. Isolated the free apps

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our data sets.

Based on our inspection on the two dataset, we conclude that:
1. we'll need to build a frequency table for the *prime_genre column* of the App Store data set, 
2. and for the *Genres* and *Category* columns of the Google Play data set.

In [48]:
def freq_table(dataset, index):
    table = {}
    total = 0
    for term in dataset:
        total +=1
        value = term[index]
        if value in table:
            table[value] +=1
        else:
            table[value] = 1
    table_percentages = {}
    
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages
            
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

display_table(free_apple_app, -5) #prime_genre


Games : 55.64595660749507
Entertainment : 8.234714003944774
Photo & Video : 4.117357001972387
Social Networking : 3.5256410256410255
Education : 3.2544378698224854
Shopping : 2.983234714003945
Utilities : 2.687376725838264
Lifestyle : 2.3175542406311638
Finance : 2.0710059171597637
Sports : 1.947731755424063
Health & Fitness : 1.8737672583826428
Music : 1.6518737672583828
Book : 1.6272189349112427
Productivity : 1.5285996055226825
News : 1.4299802761341223
Travel : 1.3806706114398422
Food & Drink : 1.0601577909270217
Weather : 0.7642998027613412
Reference : 0.4930966469428008
Navigation : 0.4930966469428008
Business : 0.4930966469428008
Catalogs : 0.22189349112426035
Medical : 0.19723865877712032


We can see that among the free English apps in the apple store, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

In [49]:
display_table(free_google_app, 1) # Category

FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 0

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [50]:
display_table(free_google_app, -4) #Genres

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.317161232088458
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051
S

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.


On the previous screen, we generated frequency tables for the columns prime_genre, Genres, and Category. We'll now focus on analyzing these frequency tables.

In [51]:
#For the prime_genre
apple_genre = freq_table(free_apple_app, -5)
for genre in apple_genre:
    total = 0
    len_genre = 0
    for app in free_apple_app:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Travel : 20216.01785714286
Productivity : 19053.887096774193
Book : 8498.333333333334
Music : 56482.02985074627
Health & Fitness : 19952.315789473683
Reference : 67447.9
Sports : 20128.974683544304
Entertainment : 10822.961077844311
Games : 18924.68896765618
Medical : 459.75
Food & Drink : 20179.093023255813
Shopping : 18746.677685950413
Catalogs : 1779.5555555555557
Utilities : 14010.100917431193
Weather : 47220.93548387097
News : 15892.724137931034
Education : 6266.333333333333
Finance : 13522.261904761905
Navigation : 25972.05
Social Networking : 53078.195804195806
Lifestyle : 8978.308510638299
Business : 6367.8
Photo & Video : 27249.892215568863


In [52]:
for app in free_apple_app:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
高德地图（精准专业的手机地图） : 1040
百度地图-智能的手机导航，公交地铁出行必备 : 1014
百度地图HD : 771
ImmobilienScout24: Real Estate Search in Germany : 187
ナビタイムの乗り換え案内 - 遅延情報やバス時刻表を案内するアプリ : 48
高德地图HD : 26
Railway Route Search : 5
NAVIRO(ナビロー) - カーナビ/バイクナビ/徒歩ナビが使える高性能ナビアプリ : 0
ホラースポット-ghost spot-意味が分かると怖いマップ : 0
MapFan(マップファン) – 渋滞情報/オービス/オフライン対応の本格カーナビ : 0
JR東日本アプリ : 0
えほう - 最強の恵方コンパス : 0
バーチャル恵方巻【節分・恵方コンパス・方位】 : 0
恵方コンパス. : 0
ナビタイム ドライブサポーター - NAVITIMEのカーナビアプリ : 0
自転車ナビ by NAVITIME(ナビタイム) - 自転車のナビができるアプリ : 0


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [54]:
for app in free_apple_app:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
彩库宝典-【官方版】 : 0
Jishokun-Japanese English Dictionary & Translator : 0
無料で音楽や写真・カメラの裏技アプリ for iPhone7 : 0



However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

#Now let's analyze the Google Play market a bit.

In [55]:
display_table(free_google_app, 5) # the Installs columns

1,000,000+ : 15.728308699086089
100,000+ : 11.55365000564143
10,000,000+ : 10.549475346947986
10,000+ : 10.199706645605326
1,000+ : 8.394448832223853
100+ : 6.916393997517771
5,000,000+ : 6.826131106848697
500,000+ : 5.562450637481666
50,000+ : 4.772650344127271
5,000+ : 4.513144533453684
10+ : 3.542818458761142
500+ : 3.2494640640866526
50,000,000+ : 2.3017037120613786
100,000,000+ : 2.1324607920568655
50+ : 1.9180864267178157
5+ : 0.7898002933543946
1+ : 0.5077287600135394
500,000,000+ : 0.270788672007221
1,000,000,000+ : 0.2256572266726842
0+ : 0.045131445334536835



One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, 
1. we'll need to convert each install number to float — this means that we need to remove the commas 
2. and the plus characters, 
3. otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [56]:
categories_android = freq_table(free_google_app, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in free_google_app:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

MAPS_AND_NAVIGATION : 4056941.7741935486
EVENTS : 253542.22222222222
COMICS : 817657.2727272727
LIFESTYLE : 1437816.2687861272
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
COMMUNICATION : 38456119.167247385
BOOKS_AND_REFERENCE : 8767811.894736841
ART_AND_DESIGN : 1986335.0877192982
BUSINESS : 1712290.1474201474
SPORTS : 3638640.1428571427
LIBRARIES_AND_DEMO : 638503.734939759
AUTO_AND_VEHICLES : 647317.8170731707
VIDEO_PLAYERS : 24727872.452830188
GAME : 15588015.603248259
TOOLS : 10801391.298666667
DATING : 854028.8303030303
HEALTH_AND_FITNESS : 4188821.9853479853
PERSONALIZATION : 5201482.6122448975
MEDICAL : 120550.61980830671
ENTERTAINMENT : 11640705.88235294
HOUSE_AND_HOME : 1331540.5616438356
FAMILY : 3697848.1731343283
PARENTING : 542603.6206896552
BEAUTY : 513151.88679245283
EDUCATION : 1833495.145631068
PRODUCTIVITY : 16787331.344927534
FOOD_AND_DRINK : 1924897.7363636363
NEWS_AND_MAGAZINES : 9549178.467741935
FINANCE : 1387692.475609756
TRAVEL_AND_LOCAL : 13984077

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [57]:
for app in free_google_app:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess


If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [59]:
under_100_m = []

for app in free_google_app:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386


We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [61]:
for app in free_google_app:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

In [63]:
for app in free_google_app: #Extreme popular apps
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


In [64]:
for app in free_google_app: #less popular apps in Books and reference
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

##conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.